In [1]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
from urllib.request import Request, urlopen
import time
from selenium.common.exceptions import NoSuchElementException

# gizbot
def getReviews(list_of_page):
    reviews = []
    for page_url in list_of_page:
        page = requests.get(page_url)    
        if page.status_code==200:   # Check if success
            soup = BeautifulSoup(page.content, 'html.parser')
            raw_header = soup.find('h1', {'class': 'articleheading'})
            header = raw_header.get_text().strip()
            raw_review = soup.find('div', {'class': 'oi-article-rt'}) 
            review = ''
            for p in raw_review.find_all('p'):
                review = review + p.get_text().strip()
            
            if 'Review' not in header:
                if 'First Impression' in header:
                    name = header.split(" First Impression")[0]
                elif 'review' not in header:
                    name = header.split(" review")[0]
                else:
                    name = header.split(":")[0]
            else:
                name = header.split(" Review")[0]

            reviews.append({ 'Name': name, 'Heading': header, 'Review': review, 
                            'Source': 'gizbot', 'URL': page_url})
    df = pd.DataFrame(reviews)
    return df

def getListOfPhone(page_url, base_url):
    links = []
    page = requests.get(page_url) # request current page
    soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
    links = fetchLinks(soup, links, base_url)
    
    next_button = soup.find('a', {'class': 'next full_width'})
    next_link = base_url + next_button.get('href')
    page = requests.get(next_link) # request current page
    soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
    links = fetchLinks(soup, links, base_url)
    
    count = 0
    while True:
        try:
            next_button = soup.find('a', {'class': 'next half_width'})
            next_link = base_url + next_button.get('href')
        except NoSuchElementException:
            break # stop after last page
        except AttributeError:
            break # stop after last page
        page = requests.get(next_link) # request current page
        soup = BeautifulSoup(page.content, 'html.parser') # parse HTML content
        links = fetchLinks(soup, links, base_url)
        count = count + 1
    return links  

def fetchLinks(soup, links, base_url):
    divs = soup.find_all('div', {'class': 'post-heading'})
    for div in divs:
        anchors = div.find('a')
        links.append(base_url + anchors.get('href'))
    return links

page_url = 'https://www.gizbot.com/mobile/reviews/'
base_url = 'https://www.gizbot.com'
list_of_page = getListOfPhone(page_url, base_url)

df = getReviews(list_of_page)
df.to_csv('CSVs/gizbot.csv')
df

,Name,Header,Review,source,link
0,Apple iPhone 13,Apple iPhone 13 Review From A Long-Term Androi...,I have used Android devices for the most part ...,gizbot,https://www.gizbot.com/mobile/reviews/apple-ip...
1,Realme GT Neo 2,Realme GT Neo 2 Review: Raw Performance Over E...,There's a new 'flagship-killer' in town and it...,gizbot,https://www.gizbot.com/mobile/reviews/realme-g...
2,Color OS 12 Hands-On,Color OS 12 Hands-On Review: What’s Coming To ...,We have been testing a very special Oppo Find ...,gizbot,https://www.gizbot.com/mobile/reviews/color-os...
3,Easyfone Shield,Easyfone Shield Review: Right Companion For Yo...,Rugged phones always have a niche for them in ...,gizbot,https://www.gizbot.com/mobile/reviews/easyfone...
4,Vivo X70 Pro+,Vivo X70 Pro+ Review: Beats The Competition Ha...,The Vivo X70 Pro+ has been my daily driver for...,gizbot,https://www.gizbot.com/mobile/reviews/vivo-x70...
...,...,...,...,...,...
722,Sony Xperia Z3+ first impressions: A beautiful...,Sony Xperia Z3+ first impressions: A beautiful...,Sony Xperia Z3+ is the company's new flagship ...,gizbot,https://www.gizbot.com/mobile/reviews/sony-xpe...
723,BlackBerry Leap,BlackBerry Leap Review: Refined Z3 with Bigger...,"Blackberry, which is still struggling to get i...",gizbot,https://www.gizbot.com/mobile/reviews/blackber...
724,InFocus M530 hands-on: Loaded with features fo...,InFocus M530 hands-on: Loaded with features fo...,InFocus entered the smartphone market in India...,gizbot,https://www.gizbot.com/mobile/reviews/infocus-...
725,LG G4,LG G4 First Impression: A Powerful Smartphone ...,LG is in an attempt to provide users with a sm...,gizbot,https://www.gizbot.com/mobile/reviews/lg-g4-fi...
